In [ ]:
import pandas as pd
import numpy as np
import keras
import training as t
import functionss  as f
import matplotlib.pyplot as plt

In [ ]:
#load csv in pandas dataframe
df = pd.read_csv('hyper_metric.csv')

In [ ]:
df.head()

In [ ]:
#get 5 highest accuracies 
df = df.sort_values(by=['Accuracy'], ascending=False)

In [ ]:
df.head(5)

In [ ]:
df = df.sort_values(by=['Precision'], ascending=False)

In [ ]:
df.head(5)

In [ ]:
df = df.sort_values(by=['AUC'], ascending=False)

In [ ]:
df.head(5)

In [ ]:
#calculate f1 score from Recall and Precision
df['F1'] = 2 * (df['Precision'] * df['Recall']) / (df['Precision'] + df['Recall'])

In [ ]:
df = df.sort_values(by=['F1'], ascending=False).head(5)

In [ ]:
df.head(5)

In [ ]:
#load model and print AUC
model = keras.models.load_model('/home/ubuntu/02_hyper/ref_model')



In [ ]:
group_label=[]
data_dir = 'data/MDD_EC'
group_label, data_MDD = t.load_data(dataDir=data_dir,group_label=group_label, sliding_window_size=16, sliding_window_overlap=0.8)

#Load and preprocess the data for Healthy
data_dir = 'data/H_EC'
group_label, data_H = t.load_data(dataDir=data_dir,group_label=group_label, sliding_window_size=16, sliding_window_overlap=0.8)

#Trim the shape of the data if necessary
#data_H, data_MDD= t.control_for_shape(data_H, data_MDD)

#make X and y data
X_data,y_data=t.makeX_Y_data(data_H, data_MDD)

#Permutate and shuffel data
X_data, y_data, group_label = t.permutate(X_data, y_data, group_label,4)

#Split data into train and test
X_train, X_test, y_train, y_test, groups_train, groups_test = t.split_data(X_data, y_data, group_label, [5,6,7,8,37,48,39,40])

#Stratified Group K Fold
#groups_train, groups_val, groups_test, X_train, X_val, X_test, y_train, y_val, y_test = t.stratified_group_k_fold(X_data, y_data, group_label)

#expand variables
expanded_X, expanded_y = t.expand_dim([X_train, X_test], [y_train, y_test])
X_train, X_test = expanded_X
y_train, y_test = expanded_y


In [ ]:
from sklearn.metrics import roc_curve
y_pred_keras = model.predict(X_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_keras)

In [ ]:
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

In [ ]:
#change color of line to green
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras), color='black')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.plot([0, 1], [0, 1], 'k--', color='red')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
# Plot the accuracy curve
y_pred_binary = (y_pred_keras >= 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred_binary)
plt.plot([0, 1], [0.5, 0.5], color='gray', linestyle='-.')
plt.scatter(1.0, accuracy, color='r', label='Accuracy ({:.2f})'.format(accuracy))
plt.legend(loc='lower right')